
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*7uD66fpC6Zp8W8spnKL1nQ.png" alt="Databricks Learning" style="width: 600">
</div>

%md
# Configurar Serverless

Antes de ejecutar el motor Serverless debes cargar las librerías

1. Clic en => Environment en el panel de la derecha
2. En la sección Dependencies carga el archivo requirements que se encuentra en la raíz del proyecto
3. Presiona apply

%md
# Generación de datos 
Al ejecutar 

```python
parameters = {"volume_path": "/volume/folder"}
dbutils.notebook.run("../Includes/GenDataStream", 3600, parameters)
```

Usted debe proporcionar la ruta del volumen donde se van a almacenar los datos generados bajo demanda (cada vez que ejecute el comando se van a generar nuevos datos)

Para poder crear un volumen siga estas instrucciones:

```sql
-- Crear un catalogo
CREATE CATALOG IF NOT EXISTS [catalogo]
-- Crear un esquema 
CREATE DATABASE IF NOT EXISTS [catalogo].[esquema]
-- Crear un volumen
CREATE VOLUME IF NOT EXISTS [catalogo].[esquema].[volumen]

```

Mediante el navegador de catálogos de Databricks ubíquese en el volumen y copie la ruta, y guardelo como valor del widget llamado volume_path

Ejemplo:

```sh
/Volumes/catalogo/esquema/volumen
```


In [0]:
dbutils.widgets.text("volume_path","")

In [0]:
volume_path=dbutils.widgets.get("volume_path")

In [0]:
dbutils.notebook.run("../Includes/GenMedallion", 3600, {"volume_path": volume_path})


## Exploring The Source dDirectory

In [0]:
files = dbutils.fs.ls(f"{volume_path}/medallion")
display(files)


## Auto Loader

In [0]:
(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{volume_path}/schema_cache")
    .load(f"{volume_path}/medallion/clientes")
    .createOrReplaceTempView("clientes_raw"))


## Enriching Raw Data

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW clientes_tmp AS (
  SELECT *, current_timestamp() arrival_time, input_file_name() source_file
  FROM clientes_raw
)

## Creating Bronze Table

In [0]:
(spark.table("clientes_tmp")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{volume_path}/checkpoints/clientes_bronze")
      .outputMode("append")
      .trigger(availableNow=True)
      .table("clientes_bronze"))

In [0]:
%sql
SELECT * FROM clientes_bronze

## Creating Silver Table

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQDatasetRule, Criticality
from databricks.labs.dqx import check_funcs
import yaml

Aplica 3 reglas de calidad de datos
- el id debe ser > 1
- El correo debe cumplir con la expresión regular
- la fecha de nacimiento no puede ser mayor a la fecha actual


Ejemplos: https://github.com/databrickslabs/dqx/blob/main/demos/dqx_manufacturing_demo.py

In [0]:
checks_from_yaml = yaml.safe_load("""
- criticality: error
  check:
    function: sql_expression
    arguments:
      expression: (id > 1)
      msg: id must have values grather than 0
      name: pk positive
      negate: false
- criticality: error
  check:
    function: regex_match
    arguments:
      column: correos
      regex: '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
- criticality: warn
  check:
    function: is_not_in_future
    arguments:
      column: fecha_nacimiento
      offset: 0
""")
ws = WorkspaceClient()
dq_engine = DQEngine(ws)
status = DQEngine.validate_checks(checks_from_yaml)
print(f"[INFO] Check rules {status}")
df_clientes = spark.table("clientes_bronze")
clientes_valid_df, clientes_invalid_df = dq_engine.apply_checks_by_metadata_and_split(df_clientes, checks_from_yaml)
display(clientes_valid_df)
display(clientes_invalid_df)

In [0]:
clientes_valid_df.write.mode("overwrite").format("delta").saveAsTable("clientes_silver")

## Creating Gold Table

Para generar la tabla de Gold, es necesario que unas todas las tablas de esta forma
- clientes_silver.id = ventas_silver.customer_id
- productos_silver.id_product = ventas_silver.id_product

La tabla resultante debe llamarse comercial_gold y deberá tener los campos:

- id_order
- customer_id,
- id_product
- nombre
- genero
- fecha_nacimiento
- correos
- nombre_producto,
- categoria
- valor_unitario
- valor_descuento
- unidades
- (valor_unitario * unidades * (1-valor_descuento)) as total

Finalmente genera una tabla con el valor total vendido por categorias


## Stopping active streams

In [0]:
for s in spark.streams.active:
    print("Stopping stream: " + s.id)
    s.stop()
    s.awaitTermination()